In [2]:

import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output
from sklearn.metrics import auc , roc_auc_score
from sklearn.model_selection import train_test_split
sys.path.append("/home/advice/Python/SR/Custom/PreProcessingUtils/")
import seaborn as sns
import re , os
from ColMatch import MatchVariable
from TabularDataPipeline import *
from pre_utils import make_dirs
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

from IPython.display import clear_output

/root/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
ResultPath = "./Result/Preprocessing_test"
folder_names = make_dirs(ResultPath , local_path=["weight_importance", "nnTree"])
ResultPath , WeightPath , nnTreePath = folder_names

In [4]:
data = pd.read_csv("./../../../Data/kdd/uci/uci_creditcard-train-0.2-0.0.csv")
in_var = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6",
          "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
          "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]
target_var = ['default payment next month']
fac_var = [ 'SEX','EDUCATION','MARRIAGE',]
num_var = [i for i in in_var if not i in fac_var]
#in_var = num_var + fac_var

In [6]:
train = data[data.sep_idx ==1 ]
valid = data[data.sep_idx ==0 ]

In [7]:
test = pd.read_csv("./../../../Data/kdd/uci/uci_creditcard-test-0.2-0.0.csv")
# test = test.drop(["ID","sep_idx"],axis=1)
# test.reset_index(drop=True ,inplace=True)

In [8]:
train.shape, valid.shape, test.shape

((21018, 26), (8982, 26), (6004, 26))

In [17]:
pipe = TabularPipeline(in_var, num_var, target_var)

please insert config type in ['Outlier', 'NumericImpute', 'FactorImpute', 'Numeric', 'Category', 'NewVar']


In [18]:
info = {
#     "Outlier" : {"method" : "IQR", "multiple_outlier_n" :4 , "var": num_var} , 
    "NumericImpute" : {"method" : "mean", "var" : num_var},
    "FactorImpute" : {"method" : "most_frequent", "var" : fac_var},
    "Numeric" : {"method" : "sklearn:minmax", "var" : num_var , 
                 "method_option" : {"feature_range" : (-1.5,1.5)}},
    "Category" : {"method" : "labelencoding", "dummy_na" : False} , 
       }

In [19]:
pipe.insert(info)

In [20]:
pipe.Steps()

{'method': 'sklearn:minmax', 'var': ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'], 'method_option': {'feature_range': (-1.5, 1.5)}}
Build PipeLine


In [21]:
pipe.fit(train)

Transform Missing Data / Method : mean
Transform Missing Data / Method : most_frequent
Transform Numeric Data Method : ('sklearn:minmax', "{'feature_range': (-1.5, 1.5)}")
Fit Category Handler
['ID', 'sep_idx', 'default payment next month']
Fitting!


In [22]:
save_pipe_path = f"{ResultPath}/pipe.pkl"
ReusePipe = BADSPipeLine(save_pipe_path)
ReusePipe.save(pipe.PIPE)
ReusePipe.load()

Save Pipeline path : ./Result/Preprocessing_test/pipe.pkl
Load Pipeline


In [23]:
train = ReusePipe.transform(train)
valid = ReusePipe.transform(valid)
test = ReusePipe.transform(test)

Transform Missing Data / Method : mean
Transform Missing Data / Method : most_frequent
Transform Numeric Data Method : ('sklearn:minmax', "{'feature_range': (-1.5, 1.5)}")
Transform Categorical Data / method : labelencoding
Transform Missing Data / Method : mean
Transform Missing Data / Method : most_frequent
Transform Numeric Data Method : ('sklearn:minmax', "{'feature_range': (-1.5, 1.5)}")
Transform Categorical Data / method : labelencoding
Transform Missing Data / Method : mean
Transform Missing Data / Method : most_frequent
Transform Numeric Data Method : ('sklearn:minmax', "{'feature_range': (-1.5, 1.5)}")
Transform Categorical Data / method : labelencoding


In [24]:
train_x ,train_y , _ = ReusePipe.split(train, numpy= True)
valid_x , valid_y , _ = ReusePipe.split(valid, numpy= True)
test_x , test_y , _ = ReusePipe.split(test, numpy= True)

In [25]:
def variable_select(in_var = None , method = None , select_n = None , NTree = None) :
    var_n = len(in_var)
    if method== "sqrt" :
        value =np.sqrt(var_n)
    elif method == "log2" :
        value =np.log2(var_n)
    elif method == "select" :
        value = select_n
    else :
        value = var_n
    output = [
        list(np.random.choice(np.arange(len(in_var)) ,
                              replace = False , size = value)) for _ in range(NTree)]
    return output

In [26]:
SELECT_N = 3
select_var = variable_select(in_var=in_var , method="select" , 
                               select_n= 17 ,NTree= SELECT_N)

In [27]:
def tf_mish(x) :
    return x * tf.nn.tanh(tf.nn.softplus(x))
activate_candidate = \
[tf.nn.selu, tf_mish , tf.nn.elu , tf.nn.relu , tf.nn.swish]

In [28]:
class LayerForEnsemble(object) :
    def __init__(self , fac_var,category_info) : 
        self.fac_var = fac_var
        self.info = category_info
    def run(self, X , select_var, name, type="embedding", emb_dim = 4) :
        inputs = []
        for idx in select_var :
            split = tf.slice(X, [0, idx], [-1, 1])
            if idx in list(self.info.keys()) :
                split =tf.cast(split, dtype=tf.int32)
                split = tf.reshape(split,(-1,))
                size = max(list(self.info[idx].values())) + 1
                if type == "embedding" :    
                    Cat = tf.keras.layers.Embedding(size,emb_dim)(split)
                elif type == "onehot" :
                    Cat = tf.one_hot(split ,depth=size)
                else :
                    raise Exception(f"No Valid Type : {type}, Please Change the type to onehot or embedding")
                inputs.append(Cat)
            else :
                inputs.append(split)
        x_input = tf.concat(inputs , axis = 1, name=name)
        return x_input
def Layer(X , hdims, name , activation,
          bn_istraining,DropoutRate,reuse = False) :
    with tf.variable_scope(f"EnsLayer_{name}",reuse=reuse):
        for idx2 , h_dim in enumerate(hdims) :
            if idx2 == 0 :
                W = tf.get_variable(f"w_{idx2}",shape=[X.get_shape()[1],h_dim])
                B = tf.get_variable(f"b_{idx2}",shape=[h_dim])
                layer = tf.matmul(X, W) + B
            else :
                W = tf.get_variable(f"w_{idx2}",shape=[hdims[idx2-1] ,h_dim ])
                B = tf.get_variable(f"b_{idx2}",shape=[h_dim])    
                layer = tf.matmul(layer, W) + B
            if len(hdims) == idx2 + 1 :
                logit = layer
            else :
                layer = tf.layers.\
                batch_normalization(layer, center=True, 
                                    scale=True, training=bn_istraining)
                layer = activation(layer)
                layer = tf.nn.dropout(layer, keep_prob=DropoutRate)
    return logit
    
def EnsembleNN(X , hidden = [[],[]], bn_istraining=None,
               DropoutRate=None, Combs=None , RepLayer= None) :
    Ensembles = []
    with tf.variable_scope("NNEnsemble") :
        for idx , Comb in enumerate(Combs) :
            x_input = RepLayer.run(X, Comb, f"nnTree{idx}","embedding",3)
            X_DIM = x_input.get_shape().as_list()[1]
            dims = hidden[idx]
            dims = [X_DIM] + dims
            print(f"No.{idx} nnTree : {str(dims)}")
            SELECT = np.random.randint(0 , len(activate_candidate) , 1)[0]
            activation = activate_candidate[SELECT]
            LAYER = Layer(x_input , dims, idx , activation,
                          bn_istraining,DropoutRate)
            Ensembles.append(LAYER)
    return Ensembles

In [29]:
row , dim = train_x.shape
print(row,dim)
target_n = len(np.unique(train_y)) 
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [ None , dim])
if target_n == 2 :
    y = tf.placeholder(tf.float32, shape = [ None , 1])
elif target_n ==1 :
    raise Exception("only one value")
else :
    y = tf.placeholder(tf.float32, shape = [ None , target_n])
DropoutRate = tf.placeholder(tf.float32, name ="dropoutRate")
training = tf.placeholder_with_default(False, shape=[], name="training")
global_step = tf.get_variable('global_step', [], initializer=tf.constant_initializer(0), trainable=False)
bn_istraining  = tf.placeholder(tf.bool)
batch_size = tf.placeholder(tf.int64)

21018 23


In [30]:
data_tuple = (X,y)
class_dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
class_dataset = class_dataset.shuffle(buffer_size= 30000,)
class_dataset = class_dataset.batch(batch_size=batch_size, drop_remainder=True)
iter = class_dataset.make_initializable_iterator()
feature_x , label_y = iter.get_next()

W0411 18:13:34.195964 140528790968064 deprecation.py:323] From <ipython-input-30-a1d5d68f924b>:5: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


In [31]:
len(select_var)

3

In [32]:
information = pipe.return_trans_info()

In [33]:
target_ny = int(y.get_shape()[1]) 
HIDDEN = [
    [ 21 , 10, 5, target_ny] , 
    [ 15 , 8, target_ny] ,
    [ 25 , 10, 5, target_ny] ,
    [ 15 , 8, target_ny] ,
    [ 25 , 10, 5, target_ny]
]
RepLayer = LayerForEnsemble(fac_var , information["cat_info"])
NModels = EnsembleNN(feature_x , hidden = HIDDEN , 
                     bn_istraining=bn_istraining, 
                     DropoutRate=DropoutRate,
                     Combs= select_var,
                     RepLayer = RepLayer
                    )

W0411 18:13:36.332198 140528790968064 deprecation.py:506] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0411 18:13:36.345810 140528790968064 deprecation.py:506] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0411 18:13:36.413512 140528790968064 deprecation.py:323] From <ipython-input-28-1b9d9821b7cb>:41: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and w

No.0 nnTree : [21, 21, 10, 5, 1]
No.1 nnTree : [23, 15, 8, 1]
No.2 nnTree : [21, 25, 10, 5, 1]


In [34]:
weights_control = tf.get_variable("weight_control",shape = [len(NModels)] , 
                                  dtype = tf.float32 , 
                                  initializer = tf.constant_initializer(1/len(NModels)),
                                 constraint= lambda x: tf.clip_by_value(x, 0.01, 0.8))

In [35]:
weights_control = tf.nn.softmax(weights_control)

In [36]:
weights_control_dim = tf.expand_dims(weights_control, axis = 1)

In [37]:
NModels_ = [tf.expand_dims(model,axis=1)  for model in NModels]

In [38]:
models = tf.concat(NModels_,axis=1) * weights_control_dim

In [39]:
logit = tf.reduce_sum(models , axis =1)

In [40]:
logit

<tf.Tensor 'Sum:0' shape=(?, 1) dtype=float32>

In [41]:
loss2 = tf.nn.weighted_cross_entropy_with_logits(
    labels = label_y , logits=logit,
    pos_weight=1.2) 

In [42]:
Probs = []
Losses = []
argMax = [ ]
for idx , Model in enumerate(NModels) :
#     loss2 = tf.nn.\
#     weighted_cross_entropy_with_logits(
#         labels = label_y , logits=tf.slice(models , [0,idx],[-1,1]),
#         pos_weight=2.0)
    PROB = tf.nn.sigmoid(Model)
    argmax = tf.where(PROB > tf.constant(0.5), 
                      tf.ones_like(PROB), tf.zeros_like(PROB))
    argMax.append(argmax)
    Probs.append(PROB)
    Losses.append(loss2)
#     Probs += tf.nn.sigmoid(Model)
#prob = tf.reshape(tf.reduce_sum(tf.concat(Probs,axis=1)*weights_control,axis=1),(-1,1))
#prob = tf.reshape(tf.reduce_sum(tf.concat(NModels,axis=1)*weights_control,axis=1),(-1,1))
prob = tf.nn.sigmoid(logit)
total_argmax = tf.where(prob > tf.constant(0.5), 
                      tf.ones_like(prob), tf.zeros_like(prob))
argMax.append(total_argmax)
# loss2 = tf.reduce_mean(Losses)

W0411 18:13:40.623680 140528790968064 deprecation.py:323] From <ipython-input-42-d403ffd19225>:11: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [43]:
argmax

<tf.Tensor 'Select_2:0' shape=(?, 1) dtype=float32>

In [44]:
argMax.append(label_y)

In [45]:
comp_pred = tf.equal( total_argmax, label_y )
accuracy = tf.reduce_mean(tf.cast(comp_pred, tf.float32))

In [46]:
auc_value , update_auc = tf.metrics.auc(label_y , prob , curve="ROC")

W0411 18:13:42.533451 140528790968064 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/metrics_impl.py:808: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [47]:
#class_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "NNEnsemble") 
totalvars = tf.trainable_variables()

In [48]:
learning_rate = tf.train.cosine_decay_restarts(1e-5, global_step,
                                               first_decay_steps=100, t_mul=1.5,m_mul=0.9, alpha=0.0)
L2= []
for v in totalvars :
    L2.append(tf.nn.l2_loss(v))
L2Regularizer= tf.add_n(L2)  * 1e-4
loss2 +=L2Regularizer
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    solver2 = tf.train.AdamOptimizer(learning_rate= learning_rate).minimize(loss2 ,var_list = totalvars)

In [49]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, TransformedBbox,
                                                   BboxPatch, BboxConnector)
def my_mark_inset(parent_axes, inset_axes, loc1a=1, loc1b=1, loc2a=2, loc2b=2, **kwargs):
    rect = TransformedBbox(inset_axes.viewLim, parent_axes.transData)
    pp = BboxPatch(rect, fill=False, **kwargs)
    parent_axes.add_patch(pp)
    p1 = BboxConnector(inset_axes.bbox, rect, loc1=loc1a, loc2=loc1b, **kwargs)
    inset_axes.add_patch(p1)
    p1.set_clip_on(False)
    p2 = BboxConnector(inset_axes.bbox, rect, loc1=loc2a, loc2=loc2b, **kwargs)
    inset_axes.add_patch(p2)
    p2.set_clip_on(False)
    return pp, p1, p2
def subplotting(ax , store , x , y ,cond={}, **kwargs) :
    ax.plot(store[x],store[y],**kwargs)
    if "ylabel" in cond :
        ax.set_ylabel(cond["ylabel"], fontsize= 10)
    if "xlabel" in cond :
        ax.set_xlabel(cond["xlabel"], fontsize= 10)
    if "xlim" in cond :
        ax.set_xlim(cond["xlim"])
    if "ylim" in cond :
        ax.set_ylim(cond["ylim"])
    if "title" in cond :
        ax.set_title(cond["title"], fontsize= 15)
    ax.legend()
    return ax
## version3
def vis_onlysl(store:dict, path:str,title:str) :
    clear_output()
    fig = plt.figure(figsize=(10, 10))
    gs = GridSpec(nrows=2, ncols=2)
    ax1 = fig.add_subplot(gs[0:1, 0])
    ax2 = fig.add_subplot(gs[0:1, 1])
    ax3 = fig.add_subplot(gs[1, 0])
    ax4 = fig.add_subplot(gs[1, 1])
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.99, 
            top=0.9, wspace=0.3, hspace=0.2)
    ax1.plot(store["epoch"],store["slloss"],label = "SLloss", color='c')
    ax1.set_xlabel('Epoch')
    select_sl= np.argmin(store["slloss"])
    ax1.vlines(store["epoch"][select_sl],
               np.min(store["slloss"]),np.max(store["slloss"]),
               label='Best', color='c')
    a =store["slloss"][-1]
    ax1.set_title(f'Loss : {a:.4f}')
    ax1.set_ylabel('Supervised', fontsize= 15)
    ax1.legend()
    ax2.plot(store["epoch"],store["auc"],label = "Train auc")
    ax2.plot(store["epoch"],store["teauc"],label = "Test auc")
    ax2.set_ylabel('AUC', fontsize= 15)
    ax2.set_xlabel('Epoch')
    select= np.argmax(store["teauc"])
    msg = f"test idx : {store['epoch'][select]}, maximum : {store['teauc'][select]*100:.2f}"
    ax2.set_title(msg)             
#     ax2.set_ylim(0.7,store['auc'][select]+0.1)
    ax2.legend()
    check_n = 4
    if len(store['epoch']) > check_n :
        axins = inset_axes(ax2, "100%", "100%", 
                           bbox_to_anchor=[0.36, .3, .5, .4],
                       bbox_transform=ax2.transAxes, borderpad=0)
        axins.plot(store['epoch'], store['auc'])
        axins.plot(store['epoch'], store['teauc'])
        maximum = np.max(store['auc'][-check_n:] + store['teauc'][-check_n:])
        minumum = np.min(store['auc'][-check_n:] + store['teauc'][-check_n:])
        xlims = (store['epoch'][-check_n],store['epoch'][-1])
        ylims = (minumum, maximum)
        axins.set(xlim=xlims, ylim=ylims)
        my_mark_inset(ax2, axins, loc1a=2, loc1b=3, loc2a=4, loc2b=4, fc="none", ec="0.5") # 
#     msg = f"Epoch : {epoch[-1]}, Loss : {loss[-1]:.3f}, Auc : {aucs[-1]:.3f}"
#     skplt.metrics.plot_ks_statistic(store["train_y"], store["train_prob"], 
#                                 ax = ax3 ,
#                                 title = "[Train] KS Static PLOT")
#     skplt.metrics.plot_ks_statistic(store["test_y"], store["test_prob"], 
#                                 ax = ax4 ,
#                                 title = "[Test] KS Static PLOT")
    sns.boxplot(x="t", y="prob", data=store["train_pd"] , ax = ax3)
    ax3.set_title("train" , fontsize= 15)
    sns.boxplot(x="t", y="prob", data=store["test_pd"] , ax = ax4)
    ax4.set_title("test" , fontsize= 15)
    plt.suptitle(title)
    plt.savefig(path)
    plt.show()
def pcolorplot(result:pd.DataFrame, save_path, title) :
    models = np.arange(result.shape[0]-2).tolist()
    models = models + ["pred","real"]
    indexs = np.arange(result.shape[1])
    fig, ax = plt.subplots( figsize = (12,8))
    plt.pcolor(s)
    plt.yticks(np.arange(0.5, len(models), 1),models)
    plt.title(title,fontsize=20)
    plt.savefig(save_path)
    plt.close()      

In [50]:
len(train_y)

21018

In [51]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
Epoch = 100000
oversample = list(np.arange(len(train_y))) + 1 * list(np.where(train_y == 1)[0])
from matplotlib.gridspec import GridSpec
from sklearn.metrics import auc , roc_auc_score
sess =  tf.Session(config=config)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)
store = {"epoch" : [] ,  "slloss" : [],
         "auc" : [], "teauc" : []}
store["train_pd"] = []
store["test_pd"] = []
i = 0

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1



In [ ]:
for i in range(i, Epoch):    
    batchSlLoss = []
    permutation_idx = np.random.permutation(oversample)
    sess.run(iter.initializer , 
             feed_dict= {X : train_x[permutation_idx],
                         y : train_y[[permutation_idx]],
                         batch_size : 300,
                        }) # switch to train dataset
    while True :
        try :
            ### Supervised Learning
            result  = sess.run([solver2 , loss2],
                               feed_dict={global_step:i, 
                                          bn_istraining :True,
                                          DropoutRate : 0.8
                                         })
            batchSlLoss.append(result[1])
        except tf.errors.OutOfRangeError :
            break
    mslloss = np.mean(batchSlLoss)
    print(f"Epoch : {i}, SLLoss : {mslloss:.6f}" , end = "\r")
    if (i % 100 == 0) :
        sess.run(iter.initializer , 
                 feed_dict= {X : train_x,
                 y : train_y,
                 batch_size : len(train_x),}) 
        tr_result  = sess.run([prob,label_y],
                              feed_dict={bn_istraining :False,
                                         DropoutRate : 1.0
                                        })
        sess.run(iter.initializer , 
                 feed_dict= {X : test_x,
                 y : test_y,
                 batch_size : len(test_x),}) 
        result  = sess.run([prob,label_y,weights_control],
                           feed_dict={bn_istraining :False,
                                      DropoutRate : 1.0
                                     })
        plt.scatter(np.arange(0,len(result[2])), result[2])
        plt.savefig(f"{WeightPath}/plot.{i:05d}.png")
        a = "Weight nnTree" +" \n"
        b = a + str(result[2])
        plt.title(b)
        plt.close()
        sess.run(iter.initializer , 
                 feed_dict= {X : test_x,
                 y : test_y,
                 batch_size : len(test_x),}) 
        dd  = sess.run(argMax+[accuracy],
                           feed_dict={bn_istraining :False,
                                      DropoutRate : 1.0
                                     })
        s = pd.DataFrame(np.concatenate(dd[:-1],axis=1)).T                
        models = np.arange(s.shape[0]-2).tolist()
        models = models + ["pred","real"]
        indexs = np.arange(s.shape[1])
        fig, ax = plt.subplots( figsize = (12,8))
        plt.pcolor(s)
        plt.yticks(np.arange(0.5, len(models), 1),models)
        plt.title(f"Epoch {i} , Auccray : {dd[-1]*100:.2f}",fontsize=20)
        plt.savefig(f"{nnTreePath}/plot.{i:05d}.png")
        plt.close()  
        store["epoch"].append(i)
        store["slloss"].append(mslloss)
        store["train_pd"] = pd.DataFrame(np.concatenate(tr_result,axis=1),
                                         columns=["prob","t"])
        store["test_pd"] = pd.DataFrame(np.concatenate(result[0:2],axis=1),
                                         columns=["prob","t"])
        store["auc"].append(roc_auc_score(store["train_pd"]["t"].values,
                                          store["train_pd"]["prob"].values))
        store["teauc"].append(roc_auc_score(store["test_pd"]["t"].values,
                                            store["test_pd"]["prob"].values))
#             store["train_prob"] =  np.concatenate((1-tr_result[2],tr_result[2]),axis=1)
#             store["test_prob"] =  np.concatenate((1-result[2],result[2]),axis=1)
        vis_onlysl(store, f"{ResultPath}/0405_SL_EnsembelNN.png",
                   title="Train Ensemblem Neural Network")
        clear_output()

In [ ]:
for key, value in  category_info.items() :
    a[key] = pd.Categorical(a[key],list(value.values()))
pd.get_dummies(a,columns=fac_var).columnsumns.tolist()

In [ ]:
pd.get_dummies(a)